In [ ]:
from src.utils import data_load
import pandas as pd
from src.s3_utils import pandas_from_csv_s3
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from utils import get_survey_question, na_rate
from hyperimpute.plugins.imputers import Imputers

import hyperimpute as hp

In [ ]:
data = data_load(data_keys={'bodyport', 'oura_activity', 'oura_sleep', "surveys"}, wave=7)


In [ ]:
df_birth = data_load(data_keys={"birth"}, wave=5)['birth']

In [ ]:
# mast_mat: shape Txd
def get_usable_window(mask, window_size=21, tolerance=2):
    result_windows = []
    T = len(mask)
    i=0
    while i < (T - window_size + 1):
        curr_window = np.array(mask[i:i+window_size])
        # usable
        if np.sum(curr_window) < tolerance:
            result_windows.append((i, i+window_size))
            i += window_size
        else:
            # add tolerance window
            tolerance_window = np.array(mask[i:i+window_size+tolerance])
            miss_idx = np.where(tolerance_window == 1)[0]
            tolerance_counter = 0
            early_stop = False
            stop_idx = None
            for j in range(len(miss_idx) - 1):
                if miss_idx[j] + 1 == miss_idx[j + 1]:
                    tolerance_counter += 1
                else:
                    tolerance_counter = 0
                if tolerance_counter >= tolerance:
                    early_stop = True
                    stop_idx = j + 1
                    # break
            if early_stop:
                i += stop_idx + 1
            else:
                # usable
                result_windows.append((i, i+window_size))
                i += window_size
    return result_windows

my_mask = [0,0,0,0,0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
print(my_mask)
get_usable_window(my_mask)
        


In [ ]:
def generate_csv_for_user(selected_user, preset_start_date=datetime(2009, 10, 12, 10, 10), preset_end_date=datetime(2030, 10, 12, 10, 10), file_name=''):

    # selected_user = 1441
    print(f"Curr user: {selected_user}")
    root_folder = file_name


    data_bodyport = data['bodyport']
    data_oura_activity = data['oura_activity']
    data_oura_sleep = data['oura_sleep']
    # data_survey = data['surveys']
    # survey_question_str = 'swollen'
    selected_data_bodyport = data_bodyport[data_bodyport['user_id'] == selected_user][['date', 
    # 'impedance_ratio', 
    # 'peripheral_fluid', 
    'impedance_mag_1_ohms', 'impedance_phase_1_degs', 
    'weight_kg']].groupby("date", as_index = False).first()
    edema_csv_path = f'/mnt/results/edema_coarse_label/user_{selected_user}_edema_coarse_label.csv'
    if not os.path.exists(edema_csv_path):
        print("Empty Edema")
        return
    selected_data_edema = pd.read_csv(edema_csv_path).groupby("date", as_index = False).first()

    selected_data_oura_sleep = data_oura_sleep[data_oura_sleep['user_id'] == selected_user][['breath_average', 'date']]
    selected_data_oura_activity = data_oura_activity[data_oura_activity['user_id'] == selected_user][[
    # 'cal_active',
    'cal_total',
    'daily_movement',
    'high',
    'inactive',
    'low',
    'medium',
    'non_wear',
    'rest',
    'steps',
    'date']]


    def get_min_date(df):
        return np.min(df['date'].astype('datetime64'))
    def get_max_date(df):
        return np.max(df['date'].astype('datetime64'))
    
    if not len(selected_data_bodyport):
        print("Empty bodyport")
        return
    if not len(selected_data_oura_activity):
        print("Empty Oura activity")
        return
    if not len(selected_data_edema):
        print("Empty Edema")
        return
    if not len(selected_data_oura_sleep):
        print("Empty Oura sleep")
        return

    overall_min_date = np.max(list(map(get_min_date, (selected_data_edema, selected_data_bodyport, selected_data_oura_activity, selected_data_oura_sleep))) + [preset_start_date])
    overall_max_date = np.min(list(map(get_max_date, (selected_data_edema, selected_data_bodyport, selected_data_oura_activity, selected_data_oura_sleep))) + [preset_end_date])
    
    date_range = pd.date_range(overall_min_date, overall_max_date, freq='d')
    print(overall_max_date-overall_min_date)
    if overall_max_date-overall_min_date < timedelta(days=10):
        return False
    date_df = pd.DataFrame()
    date_df['date'] = date_range
    date_df['date'] = date_df['date'].astype("datetime64")
    date_df.to_csv('/mnt/test2.csv')
    def change_date_type(df):
        df['date'] = df['date'].astype('datetime64')
        return pd.merge(date_df, df, how='left')

    selected_data_edema = change_date_type(selected_data_edema)
    selected_data_oura_activity = change_date_type(selected_data_oura_activity)
    selected_data_oura_sleep = change_date_type(selected_data_oura_sleep)
    selected_data_bodyport = change_date_type(selected_data_bodyport)

    unimputed_df = pd.DataFrame()
    unimputed_df['date'] = date_range
    unimputed_df = pd.merge(unimputed_df, selected_data_bodyport, how='left')
    unimputed_df = pd.merge(unimputed_df, selected_data_oura_activity, how='left')
    unimputed_df = pd.merge(unimputed_df, selected_data_oura_sleep, how='left')
    unimputed_df = pd.merge(unimputed_df, selected_data_edema, how='left')

    missingness_mask = unimputed_df.isna()
    missingness_mask = np.sum(missingness_mask, axis=1)
    missingness_mask[missingness_mask >= 1] = 1
    
    window_lst = get_usable_window(missingness_mask, tolerance=3)

    imputers = Imputers()
    imputers.list()
    method = 'hyperimpute'
    plugin = Imputers().get(method)



    if not os.path.exists(os.path.join('/', 'mnt', 'results', root_folder)):
        os.mkdir(os.path.join('/', 'mnt', 'results', root_folder))
    # save the windows
    for count, w in enumerate(window_lst):
        curr_window = unimputed_df[w[0]:w[1]]
        X = curr_window.drop('date', axis=1).drop('user_id', axis=1)
        # print(np.sum(X.isna()))
        # print(X.shape)
        curr_window = plugin.fit_transform(X.copy())
        curr_window.to_csv(f'/mnt/results/{root_folder}/user_{selected_user}_{file_name}_hyperimpute_slice{count}.csv', index=False, header=False)

        hyperimputed_df_with_date = curr_window.copy()
        hyperimputed_df_with_date['date'] = date_range[w[0]:w[1]]
        hyperimputed_df_with_date.to_csv(f'/mnt/results/{root_folder}/user_{selected_user}_{file_name}_hyperimpute_with_date_slice{count}.csv', index=False)

    return len(window_lst)



In [ ]:
counter = 0
for user in df_birth['user_id'].unique():
    user = int(user)
    if len(df_birth[df_birth.user_id == user].birth_date):
        curr_birth_date = pd.to_datetime(df_birth[df_birth.user_id == user].birth_date.values[0])
        # only interested in the 3rd trimester
        third_trimester_start_date = curr_birth_date - timedelta(days=91)
        result = generate_csv_for_user(user, preset_start_date=third_trimester_start_date, preset_end_date=curr_birth_date, file_name='edema_pred_window')
        if result:
            counter += result
counter


In [ ]:
result[1:21]

In [ ]:

selected_user = 53

curr_birth_date = pd.to_datetime(df_birth[df_birth.user_id == selected_user].birth_date.values[0])
# only interested in the 3rd trimester
third_trimester_start_date = curr_birth_date - timedelta(days=91)
a = generate_csv_for_user(selected_user, preset_start_date=third_trimester_start_date, preset_end_date=curr_birth_date, file_name='edema_pred')
np.sum(a)
get_usable_window(a, tolerance=3)


In [ ]:
for count, i in enumerate(a):
    print(count, i)

In [ ]:
# data = data_load(data_keys={'bodyport', 'oura_activity', 'oura_sleep', "surveys"}, wave=6)
data = data_load(data_keys={'bodyport'}, wave=7)
df_birth = data_load(data_keys={'birth'}, wave=5)['birth']
df_oura_activity = data_load(data_keys={"oura_activity"}, wave=7)["oura_activity"]


In [ ]:
df_garmin_activity = data_load(data_keys={"garmin_activities"}, wave=7)["garmin_activities"]
df_garmin_activity = df_garmin_activity[df_garmin_activity['record_id'] == 'sema4_524']

In [ ]:
# print(len(data['surveys']['user_id'].unique()))
print(len(data['bodyport']['user_id'].unique()))
# print(len(data['oura_sleep']['user_id'].unique()))
df_bodyport = data['bodyport']
print(len(df_bodyport))

In [ ]:
selected_bodyport_df = df_bodyport[df_bodyport['record_id'] == 'sema4_69']

In [ ]:
selected_bodyport_df.columns

In [ ]:
len(selected_bodyport_df)

In [ ]:
selected_bodyport_df.dtypes

In [ ]:
np.sum(selected_bodyport_df.isna())

In [ ]:
selected_bodyport_df = selected_bodyport_df.fillna(selected_bodyport_df.mean())                                          

In [ ]:
selected_bodyport_df.record_id

In [ ]:
selected_bodyport_df = selected_bodyport_df[[
       'backend_sway_area_mm2', 'bmi_kg_m2',
       'heart_rate', 'impedance_mag_1_ohms', 'impedance_phase_1_degs',
       'impedance_ratio', 'peripheral_fluid',
       'run_time_sec', 'sway_velocity_mm_sec', 'total_body_water_percent', 'weight_kg',]]

In [ ]:
selected_garmin_activity_df = df_garmin_activity[['active_kilocalories',
       'activity_description', 'activity_id', 
       'average_bike_cadence_in_rounds_per_minute',
       'average_heart_rate_in_beats_per_minute',
       'average_pace_in_minutes_per_kilometer',
       'average_run_cadence_in_steps_per_minute',
       'average_speed_in_meters_per_second',
       'average_swim_cadence_in_strokes_per_minute', 
       'distance_in_meters', 'duration_in_seconds', 
       'max_bike_cadence_in_rounds_per_minute',
       'max_heart_rate_in_beats_per_minute',
       'max_pace_in_minutes_per_kilometer',
       'max_run_cadence_in_steps_per_minute', 'max_speed_in_meters_per_second',
       'start_time_in_seconds', 'start_time_offset_in_seconds',
       'starting_latitude_in_degree', 'starting_longitude_in_degree', 'steps', 'total_elevation_gain_in_meters',
       'total_elevation_loss_in_meters']]

In [ ]:
# df_oura_activity[df_oura_activity['record_id'] == 'direct_10']
selected_activity_df = df_oura_activity[df_oura_activity['record_id'] == 'direct_10'][['average_met', 'cal_active',
       'cal_total', # 'class_5min', 
       'daily_movement', 
       'high', 'inactive', 'inactivity_alerts', 'low', 'medium', 'met_1min',
       'met_min_high', 'met_min_inactive', 'met_min_low', 'met_min_medium',
       'non_wear', 'rest', 'steps', 'target_calories', 'target_km',
       'target_miles', 'to_target_km', 'to_target_miles', 'total']]



In [ ]:
selected_activity_df

In [ ]:
corr = selected_bodyport_df.corr()
# corr[(corr < 0.9) & (corr > -0.9)] = 0
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, cmap=cmap, mask=mask)


In [ ]:
np.sum(df_bodyport.isna())

In [ ]:
df_bodyport[df_bodyport['user_id'] == 'user_id']

In [ ]:
def good_user(user_id, start_date, end_date, threshold=0.05, leq=True):
    selected_data_bodyport = df_bodyport[df_bodyport['user_id'] == user_id]
    # print(user)
    date_range = pd.date_range(start_date, end_date, freq='d')
    date_df = pd.DataFrame()
    date_df['date'] = date_range
    selected_data_bodyport['date'] = selected_data_bodyport['date'].astype('datetime64')
    selected_data_bodyport = pd.merge(date_df, selected_data_bodyport, how='left')
    weight_na_rate = np.max(np.sum(selected_data_bodyport['weight_kg'].isna()))/len(selected_data_bodyport)
    # print(selected_data_bodyport)
    # print(weight_na_rate)
    if leq:
        return weight_na_rate <= threshold
    else:
        return weight_na_rate >= threshold
    


In [ ]:
good_user_lst = []
bad_user_lst = []
for user in data['bodyport']['user_id'].unique():
    if len(df_birth[df_birth.user_id == user].birth_date):
        curr_birth_date = pd.to_datetime(df_birth[df_birth.user_id == user].birth_date.values[0])
        # only interested in the 3rd trimester
        start_date = curr_birth_date - timedelta(days=200)
        end_date = curr_birth_date + timedelta(days=100)
        if good_user(user, start_date, end_date, threshold=0.1):
            good_user_lst.append(user)
        if good_user(user, start_date, end_date, threshold=0.4, leq=False):
            bad_user_lst.append(user)

print(len(good_user_lst))
print(good_user_lst)
print(len(bad_user_lst))
print(bad_user_lst)

In [ ]:
df_survey = data['surveys']
survey_question_str = 'swollen'
df_edema = get_survey_question(df_survey, survey_question_str)
df_edema[df_edema['user_id'] == 293][['answer_text', 'date']]

In [ ]:
import datetime
df_birth = data_load(data_keys={'birth'})['birth']
pd.to_datetime(df_birth[df_birth.user_id == 293].birth_date.values[0])
pd.to_datetime(df_birth[df_birth.user_id == 293].birth_date.values[0]) - datetime.timedelta(days=91)

In [ ]:


df_puqe = data['surveys'].loc[data['surveys']['title']=='PUQE']
puqe_user_lst = []
for u in df_puqe['user_id'].unique():
    # 53, 54, 132, 404
    # 404: 'During the past week, did you experience any nausea?'
    # 132: 'On average in a day, for how long do you feel nauseated or sick to your stomach?'
    # 53: 'On average in a day, how many times do you vomit or throw up?'
    # 54: 'On average in a day, how many times have you had retching or dry heaves without bringing anything up?'
    selected_user_puqe = df_puqe[df_puqe['user_id'] == u]
    every_date = selected_user_puqe['date'].unique()
    count = 0
    for d in every_date:
        if len(selected_user_puqe.loc[(selected_user_puqe['date'] == d) & (selected_user_puqe['question_id'] == 404)]):
            if (selected_user_puqe.loc[(selected_user_puqe['date'] == d) & (selected_user_puqe['question_id'] == 404)]['answer_text'].iloc[0]) == 'Yes':
                count += 1
    puqe_user_lst.append((u, count))
most_user = sorted(puqe_user_lst, key=lambda x: x[1], reverse=True)[0][0]

selected_user_puqe = df_puqe[df_puqe['user_id'] == 1032]
selected_user_puqe
sorted_select = sorted(puqe_user_lst, key=lambda x: x[1], reverse=True)
list(map(lambda x: x[0], sorted_select))

# user 1032 has most puqe

In [ ]:
data['bodyport']['subsource'].unique()
# subsource is only `weight`

In [ ]:
data_bodyport = data['bodyport']
users = data_bodyport['user_id'].unique().tolist()
most_data_user = -1
most_data = -1
for u in users:
    num_data = len(data_bodyport[data_bodyport['user_id'] == u])
    if num_data > most_data:
        most_data = num_data
        most_data_user = u
most_data_user
# userid 74 has the most data for bodyport
# userid 293 has the most data for daily surveys

In [ ]:
data_bodyport.columns.tolist()

In [ ]:
selected_data_bodyport = data_bodyport[data_bodyport['user_id'] == 74][['date', 'impedance_ratio', 'peripheral_fluid', 'impedance_mag_1_ohms', 'impedance_phase_1_degs', 'weight_kg']].groupby("date", as_index = False).mean()


In [ ]:
data_oura_activity = data['oura_activity']
data_oura_activity[data_oura_activity['user_id'] == 74]
data_oura_sleep = data['oura_sleep']
# target_user_sleep = data_oura_sleep[data_oura_sleep['user_id'] == 74]
# target_user_sleep[['hr_5min', 'date']]

In [ ]:
data_oura_activity.columns.tolist()

In [ ]:
selected_data_oura_activity = data_oura_activity[data_oura_activity['user_id'] == 74][[
 'cal_active',
 'cal_total',
 'daily_movement',
 'high',
 'inactive',
#  'inactivity_alerts',
 'low',
 'medium',
 'met_min_high',
 'met_min_inactive',
 'met_min_low',
 'met_min_medium',
 'non_wear',
 'rest',
#  'rest_mode_state',
#  'score',
#  'score_meet_daily_targets',
#  'score_move_every_hour',
#  'score_recovery_time',
#  'score_stay_active',
#  'score_training_frequency',
#  'score_training_volume',
 'steps',
 'date']]

In [ ]:
def get_min_date(df):
    return np.min(df['date'])
def get_max_date(df):
    return np.max(df['date'])

In [ ]:

# # union
# overall_min_date = np.min([get_min_date(selected_data_bodyport), get_min_date(selected_data_oura_activity)])
# overall_max_date = np.max([get_max_date(selected_data_bodyport), get_max_date(selected_data_oura_activity)])

# intercept
overall_min_date = np.max([get_min_date(selected_data_bodyport), get_min_date(selected_data_oura_activity)])
overall_max_date = np.min([get_max_date(selected_data_bodyport), get_max_date(selected_data_oura_activity)])

date_range = pd.date_range(overall_min_date, overall_max_date, freq='d')
overall_df = pd.DataFrame()
overall_df['date'] = date_range
selected_data_bodyport['date'] = selected_data_bodyport['date'].astype('datetime64')
selected_data_oura_activity['date'] = selected_data_oura_activity['date'].astype('datetime64')
overall_df = pd.merge(overall_df, selected_data_bodyport, how='left')
overall_df = pd.merge(overall_df, selected_data_oura_activity, how='left')


In [ ]:
np.sum(overall_df.isna())

In [ ]:
overall_df.describe()

In [ ]:
overall_df.loc[overall_df['impedance_ratio'] > 10,'impedance_ratio'] = np.nan
np.sum(overall_df.isna())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style='darkgrid')
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.scatterplot(data=overall_df, x="date", y="non_wear")



In [ ]:
overall_df_filled = overall_df.fillna(method="ffill")
#sns.scatterplot(data=overall_df_filled, x="date", y="non_wear")
sns.scatterplot(x=overall_df_filled["date"], y=overall_df_filled["non_wear"])

In [ ]:
overall_df_filled = overall_df_filled.drop('date', axis=1)
overall_df_filled.to_csv('/mnt/results/user_72_activity_bodyport_ffilled.csv', index=False, header=False)
overall_df_without_date = overall_df.drop('date', axis=1)
overall_df_without_date.to_csv('/mnt/results/user_72_activity_bodyport_unfilled.csv', index=False, header=False)

In [ ]:
overall_df_filled.columns.to_list()

In [ ]:
len(overall_df_filled.columns.to_list())

In [ ]:
np.sum(overall_df_filled.isna())

In [ ]:
len(overall_df_filled)

In [ ]:
# hyperimpute requires python 3.7
from hyperimpute.plugins.imputers import Imputers
import numpy as np
imputers = Imputers()

imputers.list()
method = 'hyperimpute'
plugin = Imputers().get(method)
X = overall_df_without_date
out = plugin.fit_transform(X.copy())

print(method, out)

In [ ]:
hyperimputed_df_without_date = out
hyperimputed_df_without_date.to_csv('/mnt/results/user_72_activity_bodyport_hyperimpute.csv', index=False, header=False)

In [ ]:
hyperimputed_df_without_date['date'] = date_range

In [ ]:
sns.scatterplot(x=hyperimputed_df_without_date["date"], y=hyperimputed_df_without_date["impedance_ratio"], hue=overall_df['impedance_ratio'].isna())